In [1]:
import sys      
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from google.colab import drive
import tensorflow as tf
import pandas as pd
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import pandas as pd
import nltk
import re
import spacy
import sklearn.utils
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
#!pip install -U spacy
#!python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')

# Importation de la FAQ

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'
downloaded = drive.CreateFile({'id': '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'})
downloaded.GetContentFile('faq')
faq = pickle.load(open('faq', 'rb'))

texts = []

for cl,questions in faq.items() : 
  for question in questions.items():
    texts.append(str(question[0]))
    texts.append(str(question[1]))
#print(len(texts))
txt= texts

In [0]:
def lemmatise_text(text):
    """ Met à l'infinitif les verbes et met la racine des mots """
    tw_nlp = nlp(text) # spacy
    tokens = []
    for token in tw_nlp:
        tokens.append(token.lemma_)
    return ' '.join(tokens)

# Création du vectoriseur

In [5]:
import nltk
nltk.download('stopwords')
sw = nltk.corpus.stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
sw += ['être', 'avoir', 'etre', 'puis' ]

In [0]:
texts = pd.DataFrame(texts)
texts = texts[0].apply(lemmatise_text)

# Création de la matrice terme-documents

## Matrice terme documents binaire

In [0]:
vectorizer_faq_bin = CountVectorizer(binary=True,stop_words = sw)

In [9]:
vectorizer_faq_bin.fit(texts)
vectorizer_faq_bin

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [10]:
matrice_td_bin = vectorizer_faq_bin.transform(texts)
matrice_td_bin

<266x1109 sparse matrix of type '<class 'numpy.int64'>'
	with 4310 stored elements in Compressed Sparse Row format>

Exportation de la matrice termes documents

In [0]:
with open("matrice_td_bin", "wb") as fin :
  pickle.dump(matrice_td_bin, fin)

Exportation du vectorizer

In [0]:
with open("vectorizer_faq_bin", "wb") as fin :
  pickle.dump(vectorizer_faq_bin, fin)

### Matrice terme document numérique discret
nombre d'occurence

In [0]:
vectorizer_faq_num =  CountVectorizer(binary=False,stop_words = sw)

In [14]:
vectorizer_faq_num.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
matrice_td_num = vectorizer_faq_num.transform(texts)
matrice_td_num

<266x1109 sparse matrix of type '<class 'numpy.int64'>'
	with 4310 stored elements in Compressed Sparse Row format>

Exportation de la matrice termes documents

In [0]:
with open("matrice_td_num", "wb") as fin :
  pickle.dump(matrice_td_num, fin)

Exportation du vectorizer

In [0]:
with open("vectorizer_faq_num", "wb") as fin :
  pickle.dump(vectorizer_faq_num, fin)

In [0]:
vectorizer_faq_num_n_grams = CountVectorizer(min_df=5, ngram_range=(1,2), stop_words=sw).fit(texts)
matrice_td_n_grams = vectorizer_faq_num_n_grams.transform(texts)

Exportation de la matrice termes documents

In [0]:
with open("matrice_td_n_grams", "wb") as fin :
  pickle.dump(matrice_td_n_grams, fin)

Exportation du vectorizer

In [0]:
with open("vectorizer_faq_num_n_grams", "wb") as fin :
  pickle.dump(vectorizer_faq_num_n_grams, fin)

### Matrice terme document tf-idf

In [21]:
vectorizer_faq_tf_idf = TfidfVectorizer(min_df =5 , stop_words = sw)
vectorizer_faq_tf_idf.fit(texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [0]:
matrice_td_tf_idf = vectorizer_faq_tf_idf.transform(texts)

Exportation de la matrice termes documents

In [0]:
with open("matrice_td_tf_idf", "wb") as fin :
  pickle.dump(matrice_td_tf_idf, fin)

Exportation du vectorizer

In [0]:
with open("vectorizer_faq_tf_idf", "wb") as fin :
  pickle.dump(vectorizer_faq_tf_idf, fin)

# Matrice terme document binaire n-gram

Nouvelles fonctions : 

In [0]:
def substitute_url(text, url_replacement='URLEXPR'):
  txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt

def substitute_agence(text, url_replacement='AGENCE'):
  txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(tui)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(exotismes)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt 

In [26]:
nltk.download('stopwords')
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir', 'etre', 'puis','cliquer','non', 'oui', 'de']

texts = pd.DataFrame(txt)
texts = texts[0].apply(lemmatise_text).apply(substitute_url).apply(substitute_agence)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
vectorizer_faq = CountVectorizer(binary=True, ngram_range = (1,2) , stop_words = sw)
vectorizer_faq.fit(texts)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
matrice_td_faq = vectorizer_faq.transform(texts)

Exportation de la matrice termes documents

In [0]:
with open("matrice_td_faq", "wb") as fin :
  pickle.dump(matrice_td_faq, fin)

Exportation du vectorizer

In [0]:
with open("vectorizer_faq", "wb") as fin :
  pickle.dump(vectorizer_faq, fin)